In [26]:
# 예제 5-1  라이브러리 임포트
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import pandas as pd

# 예제 5-2  webdriver 실행 후 스타벅스의 지역별 매장 검색 화면에 접속
browser = webdriver.Chrome()
url = 'https://www.istarbucks.co.kr/store/store_map.do?disp=locale'
browser.get(url)


In [25]:
# 예제 5-3 webdriver로 ‘서울’ 버튼 요소를 찾아 클릭

seoul_btn_css = 'div.loca_step1_cont > ul > li:nth-child(1) > a'
seoul_btn = browser.find_element(By.CSS_SELECTOR, seoul_btn_css)
seoul_btn.click()

In [27]:
# 예제 5-4  webdriver로 ‘전체’ 버튼 요소를 찾아 클릭

all_btn_css = '#mCSB_2_container > ul > li:nth-child(1) > a'
all_btn = browser.find_element(By.CSS_SELECTOR, all_btn_css)
all_btn.click()

In [28]:
# 예제 5-5  BeautifulSoup으로 HTML 파서 만들기
html = browser.page_source
soup = BeautifulSoup(html, 'html.parser')

In [29]:
# 예제 5-6  select()를 이용해 원하는 HTML 태그를 모두 찾아오기
starbucks_soup_list = soup.select('li.quickResultLstCon')
print(len(starbucks_soup_list))

613


In [30]:
# 예제 5-7  태그 문자열 살펴보기
starbucks_soup_list[0]

<li class="quickResultLstCon" data-code="3762" data-hlytag="null" data-index="0" data-lat="37.501087" data-long="127.043069" data-name="역삼아레나빌딩" data-storecd="1509" style="background:#fff"> <strong data-my_siren_order_store_yn="N" data-name="역삼아레나빌딩" data-store="1509" data-yn="N">역삼아레나빌딩  </strong> <p class="result_details">서울특별시 강남구 언주로 425 (역삼동)<br/>1522-3232</p> <i class="pin_general">리저브 매장 2번</i></li>

In [31]:
# 예제 5-8 스타벅스 매장 정보 샘플 확인
startbucks_store = starbucks_soup_list[0]
name = startbucks_store.select('strong')[0].text.strip()
lat = startbucks_store['data-lat'].strip()
lng = startbucks_store['data-long'].strip()
store_type = startbucks_store.select('i')[0]['class'][0][4:]
address = str(startbucks_store.select('p.result_details')[0]).split('<br/>')[0].split('>')[1]
tel = str(startbucks_store.select('p.result_details')[0]).split('<br/>')[1].split('<')[0]

print(name)         # 매장명
print(lat)          # 위도
print(lng)          # 경도
print(store_type)   # 매장 타입
print(address)      # 주소
print(tel)          # 전화번호

역삼아레나빌딩
37.501087
127.043069
general
서울특별시 강남구 언주로 425 (역삼동)
1522-3232


In [32]:
# 예제 5-9 서울시 스타벅스 매장 목록 데이터 만들기
starbucks_list = []
for item in starbucks_soup_list:
    name = item.select('strong')[0].text.strip();
    lat = item['data-lat'].strip()
    lng = item['data-long'].strip()
    store_type = item.select('i')[0]['class'][0][4:]
    address = str(item.select('p.result_details')[0]).split('<br/>')[0].split('>')[1]
    tel = str(item.select('p.result_details')[0]).split('<br/>')[1].split('<')[0]
    
    starbucks_list.append( [ name, lat, lng, store_type, address, tel])

In [33]:
# 예제 5-10  pandas의 데이터프레임 생성
columns = ['매장명','위도','경도','매장타입', '주소','전화번호']
seoul_starbucks_df = pd.DataFrame(starbucks_list, columns = columns)
seoul_starbucks_df.head()

,매장명,위도,경도,매장타입,주소,전화번호
0,역삼아레나빌딩,37.501087,127.043069,general,서울특별시 강남구 언주로 425 (역삼동),1522-3232
1,논현역사거리,37.510178,127.022223,general,서울특별시 강남구 강남대로 538 (논현동),1522-3232
2,신사역성일빌딩,37.5139309,127.0206057,general,서울특별시 강남구 강남대로 584 (논현동),1522-3232
3,국기원사거리,37.499517,127.031495,general,서울특별시 강남구 테헤란로 125 (역삼동),1522-3232
4,대치재경빌딩,37.494668,127.062583,general,서울특별시 강남구 남부순환로 2947 (대치동),1522-3232


In [34]:
# 예제 5-11  데이터프레임의 요약 정보 확인
seoul_starbucks_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 613 entries, 0 to 612
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   매장명     613 non-null    object
 1   위도      613 non-null    object
 2   경도      613 non-null    object
 3   매장타입    613 non-null    object
 4   주소      613 non-null    object
 5   전화번호    613 non-null    object
dtypes: object(6)
memory usage: 28.9+ KB


In [35]:
# 예제 5-12 엑셀로 저장
seoul_starbucks_df.to_excel('./files/4_1_seoul_starbucks_list.xlsx', index=False)